In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
%matplotlib qt

In [2]:
import sys
sys.path.append('..')
import neural_ode.NeuralODE

In [3]:
tf.keras.backend.set_floatx('float64')

In [4]:
model = keras.Sequential(
    [
        keras.Input(shape=(3,)),
        layers.Dense(2, name="layer1"),
    ]
)

In [5]:
n_ode = neural_ode.NeuralODE.NeuralODE(model, 2, n_external=1)

In [6]:
n_ode.pretrain()

In [7]:
n_ode.model.variables

[<tf.Variable 'layer1/kernel:0' shape=(3, 2) dtype=float64, numpy=
 array([[-0.25860932,  0.97980293],
        [-0.60579677, -0.86430932],
        [ 0.07128016,  0.04312767]])>,
 <tf.Variable 'layer1/bias:0' shape=(2,) dtype=float64, numpy=array([0.00678932, 0.06752207])>]

In [ ]:
# Generate solution

In [9]:
#
N_n = int(2)
c = 0.4
k = 4.0
def oscilator(t, y, f):
    f_val = f(t)
    return np.array([y[1], -c*y[1]-k*y[0]+f_val])
t_final = 200.0
n_eval = int(1000)
t_span = np.array([0.0, t_final])
t_eval = np.linspace(0, t_final, num=n_eval)
y0 = np.array([1.0, 0.0])
#
f_ext = np.vstack(( np.zeros((200,1)), np.ones((400,1))*0.5, np.ones((400,1)) ) )
f_interp = interp1d(t_eval, f_ext, kind='linear', axis=0)

In [10]:
func_1 = lambda t,y: oscilator(t,y,f_interp)
sol = solve_ivp(func_1, t_span, y0, t_eval=t_eval)

<ipython-input-9-7ae9c9a7a2c5>:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array([y[1], -c*y[1]-k*y[0]+f_val])


In [11]:
plt.plot(sol.t, sol.y[0,:])

In [12]:
# transform to tensorflow
t_span_tf = tf.constant(t_span)
y0_tf = tf.constant(y0, dtype=tf.float64)
#
t_target = tf.constant(sol.t)
y_target = tf.constant(np.transpose(sol.y) )
f_ext_tf = tf.constant(f_ext)

In [13]:
# interpolation in tf
f_ext_interp_np = interp1d(t_eval, f_ext, kind='linear', axis=0)
f_ext_interp = lambda t: tf.expand_dims(tf.constant(f_ext_interp_np(t)), axis=0)

In [31]:
sol1 = n_ode.forward_solve(t_target, y_target[0,:], x_external=f_ext_interp)
fig = plt.figure()
ax = plt.gca()
ax.plot(t_target.numpy(), y_target[:,0].numpy())
ax.plot(sol1['t'].numpy(), sol1['y'][:,0].numpy())

In [ ]:
# adjoint method

In [ ]:
loss, dl_dy, a = n_ode.adjoint_method(t_target[0:3], y_target[0:3,:], x_external=f_ext_interp)

In [ ]:
# fit

In [33]:
n_epoch = 10
n_ode.fit(t_target, y_target, n_epoch=n_epoch, n_fold=5, adjoint_method=False, x_external=f_ext_tf)

--- Epoch #1 ---
Total loss of epoch: 0.004285733836240979
Elapsed time for epoch: 3.1290781497955322
--- Epoch #2 ---
Total loss of epoch: 0.004205989417755518
Elapsed time for epoch: 3.0294241905212402
--- Epoch #3 ---
Total loss of epoch: 0.004124341470287618
Elapsed time for epoch: 3.0759212970733643
--- Epoch #4 ---
Total loss of epoch: 0.004039161653622614
Elapsed time for epoch: 2.9795660972595215
--- Epoch #5 ---
Total loss of epoch: 0.00393219243301246
Elapsed time for epoch: 3.223402976989746
--- Epoch #6 ---
Total loss of epoch: 0.0038041678810714075
Elapsed time for epoch: 3.1197292804718018
--- Epoch #7 ---
Total loss of epoch: 0.003733541022143072
Elapsed time for epoch: 3.1735928058624268
--- Epoch #8 ---
Total loss of epoch: 0.003619611626390906
Elapsed time for epoch: 3.175157308578491
--- Epoch #9 ---
Total loss of epoch: 0.0036032558132319537
Elapsed time for epoch: 3.0761942863464355
--- Epoch #10 ---
Total loss of epoch: 0.0034607241736921424
Elapsed time for epoch

In [ ]:
# Check derivatives

In [35]:
n_ode.model.variables[0].assign(np.array([[0.0, -k], [1.1, -c], [0.0, 1.0]]))
n_ode.model.variables[1].assign(np.array([0,0]))

<tf.Variable 'UnreadVariable' shape=(2,) dtype=float64, numpy=array([0., 0.])>

In [36]:
#loss, dl_dp = n_ode.usual_method(t_target, y_target, x_external=f_ext_interp)
loss, dL_dy, a = n_ode.adjoint_method(t_target, y_target, x_external=f_ext_interp)
dl_dp = a[0, 2:]

In [37]:
dp = 0.00001
n_ode.model.variables[0].assign(np.array([[0.0, -k+dp], [1.1, -c], [0.0, 1.0]]))
n_ode.model.variables[1].assign(np.array([0,0]))

<tf.Variable 'UnreadVariable' shape=(2,) dtype=float64, numpy=array([0., 0.])>

In [38]:
loss2, dl_dp2 = n_ode.usual_method(t_target, y_target, x_external=f_ext_interp)

In [39]:
(loss2-loss)/dp

<tf.Tensor: shape=(), dtype=float64, numpy=-0.010174699127674103>

In [40]:
dl_dp

<tf.Tensor: shape=(8,), dtype=float64, numpy=
array([ 4.33630119e-03, -1.00475903e-02,  3.60513989e-02,  8.59889971e-03,
        5.37464028e-04,  3.81110847e-05, -4.06164777e-03, -2.54769737e-04])>